In [ ]:
%matplotlib inline
import pandas as pd
import requests
import itertools
from matplotlib import pyplot as plt
import seaborn as sns
import random
import keyring
from io import BytesIO
from sklearn.preprocessing import StandardScaler,MaxAbsScaler, MinMaxScaler
import numpy as np

In [ ]:
leagueID=keyring.get_password('ESPN','League')
w=4

Create api url for pulling all information related to fantasy scoring

In [ ]:
scoreboard=requests.get('http://fantasy.espn.com/apis/v3/games/ffl/seasons/2020/segments/0/leagues/{}?view=mBoxscore&scoringPeriodId={}&matchupPeriodId={}'.format(leagueID,w,w))

sc_data=scoreboard.json()

Generate all the mapping we will need for our blog post and to create our data

In [ ]:
pos_dict={0:'QB',2:'RB',4:'WR',6:'TE',23:'FLEX',16:'D/ST',17:'K',20:'BNCK'}

Rs={0:1,2:2,4:2,6:1,23:1,16:1,17:1}

team_map={itm['id']:itm['location']+' '+itm['nickname'] for itm in sc_data['teams']}
iteam_map = {v: k for k, v in team_map.items()}

divs={0:'East',1:'West',2:'Mid'}

team_div={x['id']:divs.get(x['divisionId']) for x in sc_data['teams']}

record_dict={itm['id']:itm['record'] for itm in sc_data['teams']}

rtxt_dict={k:'({},{},{})'.format(v['overall']['wins'],
                       v['overall']['ties'],
                       v['overall']['losses']) for k,v in record_dict.items()}

Iterate over all the matches and for each get the elegible scores. We also iterate over all possibilities for scoring to get the best team score and all others

In [ ]:
gscores={}
gno=1
for s in sc_data['schedule']:
    if s['matchupPeriodId']==w:
        print('---------')
        print("home: {} away {}".format(s['home']['teamId'],s['away']['teamId']))
        print('---------')
        tdict={}
        for ttype in ['home','away']:

            team=s[ttype]
            print(ttype.upper())
            aroster=team['rosterForMatchupPeriod']
            roster=team['rosterForCurrentScoringPeriod']
            lp=[]
            for e in roster['entries']:
                pscore=e['playerPoolEntry']['appliedStatTotal']
                pfname=e['playerPoolEntry']['player']['fullName']
                pslots=e['playerPoolEntry']['player']['eligibleSlots']
                pslot=e['lineupSlotId']
                print("{:.2f} {} Slot:{}".format(pscore,pfname,pslot),pslots)
                lp.append([pscore,pfname,pslot,pslots])

            lpd=pd.DataFrame(lp,columns=['Score','Name','Slot','Slots'])
            lpd['SlotPos']=lpd.Slot.map(pos_dict)
            lpd.SlotPos.fillna('Unknown',inplace=True)
            posln={}

            for k in Rs:
                posln[k]=lpd[lpd.Slots.apply(lambda x:k in x)].index.values

            tarr=[]
            for i,k in Rs.items():
                for ii in range(0,k):
                    tarr.append(list(posln[i]))

            lpscore=[]
            for lpos in itertools.product(*tarr):
                if len(set(lpos))==9:
                    lpscore.append(lpd.loc[list(lpos)].Score.sum())
            
            tdict[ttype]={'score':roster['appliedStatTotal'],'roster':lpd,'teamid':team['teamId'],'pos':lpscore}
        gscores[gno]=tdict
        gno=gno+1

Generate all the information we will need to create the different headings and different plots per game

In [ ]:
headings=[]
posts=[]
sns.set_theme(style="ticks",font_scale=1.2, color_codes=True)
tds=pd.DataFrame([],columns=['team'])
all_rosters=pd.DataFrame([])
for i,g in gscores.items():
    print('GAME '+str(i))
    home=team_map.get(g['home']['teamid'])
    away=team_map.get(g['away']['teamid'])
    home_score=g['home']['score']
    away_score=g['away']['score']
    

    home_n=team_map.get(g['home']['teamid'])
    home_div=team_div.get(g['home']['teamid'])
    home_rec=rtxt_dict.get(g['home']['teamid'])

    away_n=team_map.get(g['away']['teamid'])
    away_div=team_div.get(g['away']['teamid'])
    away_rec=rtxt_dict.get(g['away']['teamid'])
    
    heading="{} {} {} Div.-:{:.1f} vs {} {} {} Div.-:{:.1f} \n".format(home_n,
                                                          home_rec,
                                                          home_div,
                                                          home_score,
                                                          away_n,
                                                          away_rec,
                                                          away_div,
                                                          away_score)
    
    headings.append(heading)

    print('{}: {:.1f} vs {}: {:.1f}'.format(home,home_score,away,away_score))
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=False,figsize=(15,5))
    

    sns.histplot(g['home']['pos'],ax=ax1,bins=15,stat="probability")
    ax1.axvline(g['home']['score'],color='r')
    ax1.set_title('{}:{:.1f}'.format(home,home_score))
    

    sns.histplot(g['away']['pos'],ax=ax2,bins=15,stat="probability")
    ax2.axvline(g['away']['score'],color='r')
    ax2.set_title('{}: {:.1f}'.format(away,away_score))
    
    td=pd.DataFrame(g['home']['pos'],columns=['score'])
    td.insert(column='team',value=home,loc=0)
    td.insert(column='sc',value=home_score,loc=0)
    tds=tds.append(td)
    all_rosters=all_rosters.append(g['home']['roster'])
    
    td=pd.DataFrame(g['away']['pos'],columns=['score'])
    td.insert(column='team',value=away,loc=0)
    td.insert(column='sc',value=away_score,loc=0)
    tds=tds.append(td)
    plt.savefig('Week{}/g{}_prob.png'.format(w,i))
    
    post={
        'heading':'##'+heading,
        'img_org':'Week{}/g{}_prob.png'.format(w,i)
    }
    posts.append(post)
    all_rosters=all_rosters.append(g['away']['roster'])
    
    
    

Get the overall plots for posts

In [ ]:
fb,axbp = plt.subplots(figsize=(15,10))

sns.boxenplot(x="team", y='score', data=tds, ax=axbp)
sns.stripplot(data=tds[['team','sc']].drop_duplicates(), x="team", y="sc",color='r',jitter=0,size=10)
axbp.tick_params(axis='x', labelrotation=45)
axbp.set_title('Team Possibile Scores Week {}'.format(w));
axbp.set(xlabel='Team', ylabel='Possible Score');
plt.savefig('Week{}/overall_prob.png'.format(w))


Generate different indicators live MVP, LVP, best manager, worst manager, etc

In [ ]:
bsc=tds.groupby('team').max()
bsc.columns=['Actual Score','Best Possible Score']
bsc.index.name=''
bsc.to_markdown()

In [ ]:
bmax=bsc.idxmax().to_frame()
bmin=bsc.idxmin().to_frame()

In [ ]:
perf=[]
for gname, gp in all_rosters[['Name','Score','SlotPos']][~all_rosters['SlotPos'].isin(['Unknown','BNCK'])].groupby('SlotPos'):
    sc=StandardScaler()
    gp=gp.set_index('Name')
    xs=sc.fit_transform(gp['Score'].values.reshape(-1, 1))
    gp['Scale']=xs
    pmin=gp.loc[gp.Score.idxmin()]
    pmax=gp.loc[gp.Score.idxmax()]
    perf.append([pmax.name,pmin.name,pmax.Scale,pmin.Scale])
perfdf=pd.DataFrame(perf,columns=['Max_Name','Min_Name','MaxN','MinN'])
LVP=perfdf.loc[perfdf.MinN.idxmin].Min_Name
MVP=perfdf.loc[perfdf.MaxN.idxmax].Max_Name

In [ ]:
f = open('quotes.txt', 'r')
txt = f.read()
lines = txt.split('\n.\n')
rand=random.choice(lines)

In [ ]:
x = bsc.transpose().values #returns a numpy array
max_scaler =MaxAbsScaler()
x_scaled = max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled,columns=bsc.transpose().columns)

In [ ]:
res=df.transpose()
res['D']=res[1]-res[0]

In [ ]:
worst_manager=res.loc[res.D.idxmax].name
best_manager=res.loc[res.D.idxmin].name
worst_manager

Iterate to find the projected scores for each team

In [ ]:
gnum=1
proj_scores={}
games={}
diffs={}
all_ros_p=pd.DataFrame([])
for game in sc_data['schedule']:
    if game['matchupPeriodId']==w:
        home=game['home']
        home_n=team_map.get(game['home']['teamId'])
        home_div=team_div.get(game['home']['teamId'])
        home_rec=rtxt_dict.get(game['home']['teamId'])
        away=game['away']
        away_n=team_map.get(game['away']['teamId'])
        away_div=team_div.get(game['away']['teamId'])
        away_rec=rtxt_dict.get(game['away']['teamId'])
        
        columns=['Player','AScore','PScore','Slot','Pos','PSlots']

        plines=[]
        for ent in home['rosterForCurrentScoringPeriod']['entries']:
            a_score=ent['playerPoolEntry']['appliedStatTotal']
            for x in ent['playerPoolEntry']['player']['stats']:
                if x['statSourceId']==1:
                    pl_proj=x
                    p_score=pl_proj['appliedTotal']
                    break
            pline=[ent['playerPoolEntry']['player']['fullName'],
             a_score,
             p_score,
             ent['lineupSlotId'],
             pos_dict.get(ent['lineupSlotId'],'Unknown'),
             ent['playerPoolEntry']['player']['eligibleSlots']]
            plines.append(pline)
        tlp=pd.DataFrame(plines,columns=columns)
        tlp.insert(column='tid',loc=0,value=home['teamId'])
        tlp.insert(column='gid',loc=0,value=gnum)
        home_pscore=tlp[~tlp['Slot'].isin([20,21])].PScore.sum()
        all_ros_p=all_ros_p.append(tlp)
        
        plines=[]
        for ent in away['rosterForCurrentScoringPeriod']['entries']:
            a_score=ent['playerPoolEntry']['appliedStatTotal']
            for x in ent['playerPoolEntry']['player']['stats']:
                if x['statSourceId']==1:
                    pl_proj=x
                    p_score=pl_proj['appliedTotal']
                    break
            pline=[ent['playerPoolEntry']['player']['fullName'],
             a_score,
             p_score,
             ent['lineupSlotId'],
             pos_dict.get(ent['lineupSlotId'],'Unknown'),
             ent['playerPoolEntry']['player']['eligibleSlots']]
            plines.append(pline)
        tlp=pd.DataFrame(plines,columns=columns)
        tlp.insert(column='tid',loc=0,value=away['teamId'])
        tlp.insert(column='gid',loc=0,value=gnum)
        away_pscore=tlp[~tlp['Slot'].isin([20,21])].PScore.sum()
        
        all_ros_p=all_ros_p.append(tlp)
        
        proj_scores[home_n]=home_pscore
        proj_scores[away_n]=away_pscore
        #print(heading)
        gnum=gnum+1
        #print(tlp)

For each score compare projected vs actual, this gives us the Team that overperformed their projection and the one that underperformed the projection. Scale it so we can directly compare.

In [ ]:
bsc['Projected Score']=bsc.index.map(proj_scores)

In [ ]:
x = bsc[['Projected Score']].transpose().values
sc=bsc.transpose()

In [ ]:
sc2=sc.loc['Projected Score'].values

In [ ]:
scaler = MaxAbsScaler()
scaler.fit(x)
xsc2=scaler.transform(bsc[['Actual Score']].transpose().values)

In [ ]:
bsc['Scale_Over_Projection']=list(xsc2.flatten())

In [ ]:
Rudy=bsc.loc[bsc['Scale_Over_Projection'].idxmax].name
MH=bsc.loc[bsc['Scale_Over_Projection'].idxmin].name

Save the post to a folder that also contains the plots we made. We can just upload that to our blog(ghost)

In [ ]:
filename='Week{}/stat_post.md'.format(w)
with open(filename, 'w', encoding='utf-8') as f:
    f.write("# Week {} Stats \n".format(w))
    f.write("> {}  \n".format(rand))
    f.write('## Team Performance \n')
    f.write('Team with the highest score: {}  \n'.format(bmax.loc['Actual Score'][0]))
    f.write('Team with the highest best score: {}  \n'.format(bmax.loc['Best Possible Score'][0]))
    f.write('Team with the lowest score: {}  \n'.format(bmin.loc['Actual Score'][0]))
    f.write('Team with the lowest best score: {}  \n\n'.format(bmin.loc['Best Possible Score'][0]))
    f.write(bsc.to_markdown())
    f.write('\n\n')
    f.write('Week {} MVP: {}  \n'.format(w,MVP))
    f.write('Week {} LVP: {}  \n'.format(w,LVP))
    f.write('### Awards \n')
    f.write('Charlie Weis Award: {} \n'.format(worst_manager))
    f.write('Green-Seer Award: {} \n'.format(best_manager))
    f.write('Rudy Award: {} \n'.format(Rudy))
    f.write('Most Harmless Award: {} \n'.format(MH))
    
    for h in headings:
        f.write('## {} \n'.format(h))
    f.write(' \n')
    f.write('## Overall Team Performance \n')'''